# Introduction
<font color='orange'>[Google Colab]</font> In Part I, we explored the traffic image API and mapped out the locations of the cameras in the country.

We then acquired images from a few traffic cameras for inspection.

In this Part, this is what you'll do:
1. Import libraries
2. Collect two years' worth of traffic image JSON
3. Combine the eventual DataFrame
4. Filter for camera ID 1709 only
5. Export DataFrame for Part III

<strong>Apart from coding, data collection alone will take <font color='red'>2 hours or so</font>. Make sure you allocate plenty of time for this Part.</strong>

### Step 1: Import libraries
We'll start off with importing the libraries that we need:
- requests
- pandas as pd
- datetime
- time

In [1]:
# Step 1: Import libraries
import requests
import pandas as pd
import datetime
import time

# Testing collection
Before we collect data en masse, we should collect the data bit by bit first and measure the time taken to assess the right strategy.

We'll start by making simple class first, then building up for more while keeping an expectation of how much time these tasks will take.

### Step 2: Create a range of dates and time
We'll be retrieving a day's worth of data first, with an interval of 1 minute. 

We do that because the API documentation recommends so. Use pandas' date range to get a range of time between "2019/01/01" and "2019/01/02", with a frequency of 1 minute.

![DataRangeExample](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectComputerVisionTraffic/DataRangeOneDay.png)

Your list will be 1,441 items long.

In [2]:
# Step 2: Get a range of dates and time
DtRange=pd.date_range(start='2019-01-01', end='2019-01-02', freq='min')

In [3]:
# Optional: Check length of list
len(DtRange)

1441

### Step 3: Format the datetime into a string
Now that you have a list of datetime items, you'll have to structure them into the right format that you can use. 

You'll have to call the strftime method, along with the right format to format the eventual string output.

In this Step, try to print a string from the first datetime in the list.

```
2019-01-01 00:00:00 

to 

2019-01-01T00%3A00%3A00
```

Something like this.

In [4]:
# Step 3: Format the first datetime in list to string
DtRange[0].strftime('%Y-%m-%dT%H%%3A%M%%3A%S')

'2019-01-01T00%3A00%3A00'

### Step 4: Call API for the list of datetimes
Now that you've figured out how to format the datetime into a string, it's time to loop through them and make repeated calls and store the JSON responses in a list.

You should use the <strong>time</strong> library to measure how long it takes to call the API to get the all 1,441 times.

You should end up with a list containing 1,441 JSON objects.

<font color = 'red'>Allocate 30-40 minutes for this task.</font>

<details>
  <summary>Click here once if you're unsure and need pseudocode</summary>
  <ol>
    <li>Declare a variable containing an empty list</li>
    <li>Declare a variable containing a base URL for the API</li>
    <li>Use a for loop for the list you got from Step 3. In each loop:</li>
    <ol>
      <li>Declare a temporary URL consisting of the base URL and the current datetime in list</li>
      <li>Make a GET request using the URL and store the response in a variable</li>
      <li>Perform a .json method and store the JSON object in a new variable</li>
      <li>Append the variable into the list the you declared earlier</li>
    </ol>
  </ol>
</details>

In [5]:
# Step 4: Call API and get a list of JSON objects

JSONobjects = []
baseURL = "https://api.data.gov.sg/v1/transport/traffic-images?date_time="
for i in DtRange:
  tempURL = baseURL+i.strftime('%Y-%m-%dT%H%%3A%M%%3A%S')
  request=requests.get(tempURL)
  print(i, request)
  JSONresult=request.json()
  JSONobjects.append(JSONresult)

2019-01-01 00:00:00 <Response [200]>
2019-01-01 00:01:00 <Response [200]>
2019-01-01 00:02:00 <Response [200]>
2019-01-01 00:03:00 <Response [200]>
2019-01-01 00:04:00 <Response [200]>
2019-01-01 00:05:00 <Response [200]>
2019-01-01 00:06:00 <Response [200]>
2019-01-01 00:07:00 <Response [200]>
2019-01-01 00:08:00 <Response [200]>
2019-01-01 00:09:00 <Response [200]>
2019-01-01 00:10:00 <Response [200]>
2019-01-01 00:11:00 <Response [200]>
2019-01-01 00:12:00 <Response [200]>
2019-01-01 00:13:00 <Response [200]>
2019-01-01 00:14:00 <Response [200]>
2019-01-01 00:15:00 <Response [200]>
2019-01-01 00:16:00 <Response [200]>
2019-01-01 00:17:00 <Response [200]>
2019-01-01 00:18:00 <Response [200]>
2019-01-01 00:19:00 <Response [200]>
2019-01-01 00:20:00 <Response [200]>
2019-01-01 00:21:00 <Response [200]>
2019-01-01 00:22:00 <Response [200]>
2019-01-01 00:23:00 <Response [200]>
2019-01-01 00:24:00 <Response [200]>
2019-01-01 00:25:00 <Response [200]>
2019-01-01 00:26:00 <Response [200]>
2

### Step 5: Combine all the JSON into a DataFrame
With your list of JSON objects, turn each of them into DataFrame and append them in another list. 

![ConcatenatedDataFrameOneDay](https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectComputerVisionTraffic/ConcatenatedDataFrameOneDay.png)

With the list of DataFrames, concatenate them so that they'll end up as a single DataFrame.

You are anticipating:
- 124,874 rows
- 8 columns

In [6]:
# Step 5: Combine the list of JSON into a DataFrame
from functools import reduce

ListDF=[]
for i in JSONobjects:
  tempDF=pd.json_normalize(i['items'][0]['cameras'])
  ListDF.append(tempDF)

CombinedDF=reduce(lambda df1, df2: df1.append(df2), ListDF)

### Step 6: Filter the DataFrame for camera 1709 only
Your huge DataFrame currently contains the URLs for the images from ALL cameras in a single day.

Filter the DataFrame to contain only rows from <strong>camera_id = 1709</strong> so we can see how many rows there are in a single day. 

We should expect:
- 1441 rows
- 8 columns

In [7]:
# Step 6: Filter for 1709
FilteredDF=CombinedDF[CombinedDF['camera_id']=='1709']

In [8]:
FilteredDF

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
11,2018-12-31T23:59:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,48f34eaa179e51989c8f937631da1bce
4,2019-01-01T00:00:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,ed9a57e100a83156605b19ec84bdc2fe
8,2019-01-01T00:01:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c2713943d69482eaa4191454c76af1c8
8,2019-01-01T00:01:25+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,c2713943d69482eaa4191454c76af1c8
11,2019-01-01T00:03:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,7892491f53adea153a2345de7d5e6873
...,...,...,...,...,...,...,...,...
11,2019-01-01T23:55:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,df9921dd797b6f470ccc4271c609667c
11,2019-01-01T23:55:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,df9921dd797b6f470ccc4271c609667c
6,2019-01-01T23:57:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,3ce2dc2e7827af15b35c260ec5dc0753
18,2019-01-01T23:57:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,3ce2dc2e7827af15b35c260ec5dc0753


### Step 7: Drop duplicates based on 'image'
If you observe carefully, there are some rows that are repeated.

Drop duplicates by 'image' column and see how many rows you end up with.

In [9]:
# Step 7: Drop duplicates by 'image' column
FilteredDF.drop_duplicates(subset='image', inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Full data collection
Now that we have figured out how to collect one day's worth of data, we can now collect two one-month data:
- 2019/01/01 to 2019/02/01
- 2020/01/01 to 2020/01/01

Wait a minute. Do you notice something is off?

<details>
  <summary>Click here to check if your guess is right</summary>
  <p><strong>If doing one day's worth of API calling took 30 minutes, doing 2 months' worth of API calling means it will take 30 mins per day's JSON x 30 days per month x 2 months = 1800 minutes.</strong></p>
  <p><strong>That's 30 hours!</strong></p>
</details>

Don't worry - we'll be using multithreading to make multiple calls at the same time.

Reading: https://docs.python.org/3/library/concurrent.futures.html (scroll to ThreadPoolExecutor Example)

### Step 8: Set up date_ranges
Let's just set up the range of dates we need to collect data on.

Declare two variables containing the two date ranges we want, in minute frequency.



In [10]:
# Step 8a: Set up date range for 2019
Range2019=pd.date_range(start='2019-01-01', end='2019-02-01', freq='min')

In [11]:
# Step 8b: Set up date range for 2020
Range2020=pd.date_range(start='2020-01-01', end='2020-02-01', freq='min')

### Step 9: Define retrieveCameraJSON
Define a function nalled retrieveCameraJSON where it takes in one argument - url.

This function will return one JSON from one URL. 

In short, it's doing what you did in Step 4 but no for loop.

<details>
<summary>Click here for pseudocode if you need help</summary>
  <ol>
    <li><strong>Define</strong> retrieveCameraJSON, taking in one argument called date_time</li>
    <li>In the function definition</li>
    <ul>
    <li>Declare a variable that contains the base URL, just before date_time=</li>
    <li>Declare a variable that takes the base URL and combines it with the formatted date_time argument (refer to Step 3)</li>
    <li>Make a GET request to get the response of the API call</li>
    <li>Declare a varible to store the JSON data of the response</li>
    <li><strong>Return</strong> the variable</li>
    </ul>
  </ol>
</details>

In [40]:
# Step 9: Define retrieveCameraJSON

def retrieveCameraJSON(dt):
  baseURL = "https://api.data.gov.sg/v1/transport/traffic-images?date_time="
  tempURL = baseURL+dt.strftime('%Y-%m-%dT%H%%3A%M%%3A%S')
  request=requests.get(tempURL)
  if request.status_code==200:
    JSONresult=request.json()
    return JSONresult
  else:
    print(dt, request.status_code)

### Step 10: Import library
Since we're doing concurrency, we'll need:
- futures from concurrent

In [13]:
# Step 10: Import futures
from concurrent import futures

### Step 11: Use concurrency to retrieve JSON data from 2019
Now that we have defind retrieveCameraJSON, let's make our API calls.

With reference to the <a href="https://docs.python.org/3/library/concurrent.futures.html">reading</a> provided above, you'll make lots of requests concurrently.

If this is the first time doing a concurrency call, don't worry - try:
1. Small number of rows to see if you got it right
2. Adapting the code from the reading

<font color='red'><strong>Reserve around 60 minutes for this Step.</strong></font>

<details>
<summary>Click here once for the pseudocode if you're stuck</summary>
  <ol>
    <li>Declare an empty list to store the <strong>.result</strong> of your completed futures</li>
    <li>Use a <strong>with</strong> statement with futures.ThreadPoolExecutor, with a max_workers of 150 as an <strong>executor</strong></li>
    <li>Declare a variable, where it is a list containing the the futures of retrieveCameraJSON with the date <strong>for</strong> the date in the date range in 2019</li> 
    <li>Use a <strong>for</strong> loop for the .as_completed list of futures</li>
    <ul>
      <li>Append the .result() of each future in the list to the list that you declared at the top</li>
    </ul>
  </ol>
</details>

<br>

<details>
    <summary><font color = 'green'>SPOILERS! Click once for a redacted code block if you're really really really stuck.</font></summary>
    <div>
        <img src = 'https://uplevelsg.s3-ap-southeast-1.amazonaws.com/ProjectComputerVisionTraffic/ConcurrentFunctionCalls.png'>
    </div>
</details>

In [41]:
# Step 11a: Make concurrent API calls for 2019 data
data2019 = []
with futures.ThreadPoolExecutor(max_workers=150) as executor:
  futureJSON = [executor.submit(retrieveCameraJSON, dt) for dt in Range2019]
  for future in futures.as_completed(futureJSON):
    data2019.append(future.result())

In [42]:
# Step 11b: Get length of list of JSON results
len(data2019)

44641

### Step 12: Repeat Step 5-7 to get 2019 Jan DataFrame
Now that you have a long list of JSON objects, it's time to turn them into a DataFrame. 

Repeat what you did earlier and get a huge DataFrame containing traffic image URLs for 2019 Jan.

You should expect around <strong>30,000 rows<strong> after dropping duplicates and filtering for 1709 in the end.

In [43]:
# Step 12: Get the 2019 Jan DataFrame
from functools import reduce

List2019=[]
for i, row in enumerate(data2019):
  try:
    tempDF=pd.json_normalize(row['items'][0]['cameras'])
    FilteredDF=tempDF[tempDF['camera_id']=='1709']
    List2019.append(FilteredDF)
  except Exception as error:
    print(i, error)

1876 'camera_id'
1907 'camera_id'
2584 'camera_id'
5926 'camera_id'
16001 'camera_id'
26099 'camera_id'
36205 'camera_id'


In [49]:
Combined2019=reduce(lambda df1, df2: df1.append(df2), List2019)
Combined2019.drop_duplicates(subset='image', inplace=True)

In [50]:
Combined2019

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
10,2019-01-01T17:45:13+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,5f8f60eee6f8e0efb367a2d7fb5e821d
5,2019-01-01T07:16:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,192f8ca9dacd707f8d313500eb334a28
4,2019-01-01T02:39:12+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,43a52ef9bd8cca7856a3b99c9c2cda4f
16,2019-01-01T14:43:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,44af23fbe82436dd93fc3b5069183d73
12,2019-01-01T12:59:40+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d6a29c6a85fc6bb7eef769e8eb74fc6f
...,...,...,...,...,...,...,...,...
11,2019-01-31T23:27:46+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,95bffe517f74b40bb14a1e24a93e3408
6,2019-01-31T23:42:26+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,cecf29b479e08eaee4195b11a6c05331
11,2019-01-31T23:20:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9c21264544e37ffd894a4629ad369a43
18,2019-01-31T23:14:45+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bbf37f2ba7e45e4be9f133800cc805ff


### Step 13: Sort the DataFrame by timestamp
Using concurrency meant that the JSON in the list is not ordered chronologically.

As such, you'll have to sort the DataFrame by timestamp.

In [52]:
# Step 13: Sort the DataFrame by timestamp
Combined2019.sort_values(by = 'timestamp', ascending=True, inplace=True)

### Step 14: Export the DataFrame into a CSV file
Once you're done with sorting, export your hard work into a CSV file in your Google Drive.

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
# Step 14: Export the DataFrame into CSV
Combined2019.to_csv('/content/drive/MyDrive/Project CV x Traffic/Data2019.csv', index=None)

### Step 15: Repeat Steps 11-14 for date range in 2020
Now that you're done with 2019, time to collect data for 2020 Jan.

Make the same concurrenct calls but with the 2020 date range that you set up earlier.

You might get a different number depending on the API's behaviour, but expect around 29,900 rows at the end after removing duplicate entries.

<font color='red'><strong>Allocate another hour for this Step.</strong></font>

In [56]:
# Step 15: Repeat Steps 11-14 for 2020 data
data2020 = []
with futures.ThreadPoolExecutor(max_workers=150) as executor:
  futureJSON = [executor.submit(retrieveCameraJSON, dt) for dt in Range2020]
  for future in futures.as_completed(futureJSON):
    data2020.append(future.result())

In [61]:
len(data2020)

44641

In [57]:
List2020=[]
for i, row in enumerate(data2020):
  try:
    tempDF=pd.json_normalize(row['items'][0]['cameras'])
    FilteredDF=tempDF[tempDF['camera_id']=='1709']
    List2020.append(FilteredDF)
  except Exception as error:
    print(i, error)

4530 'camera_id'
4532 'camera_id'
14622 'camera_id'
24698 'camera_id'


In [58]:
Combined2020=reduce(lambda df1, df2: df1.append(df2), List2020)
Combined2020.drop_duplicates(subset='image', inplace=True)

In [59]:
Combined2020.sort_values(by = 'timestamp', ascending=True, inplace=True)

In [63]:
Combined2020

,timestamp,image,camera_id,location.latitude,location.longitude,image_metadata.height,image_metadata.width,image_metadata.md5
12,2019-12-31T23:59:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,1cc0aed7f981fccabfe22c43229a48d5
5,2020-01-01T00:00:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,9daa712d0a50f989a8a698b6fc7dd33b
6,2020-01-01T00:01:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d623bfdab733b966ade6e390f18cd7fb
9,2020-01-01T00:02:44+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,0ddb5b38083ebcce87c8228b746a7347
6,2020-01-01T00:03:24+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,fc4923fc71ec9a99940007cea41ecf23
...,...,...,...,...,...,...,...,...
8,2020-01-31T23:53:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,bb50d5020f94a869583863593c0308d8
16,2020-01-31T23:54:50+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,f54b2765d796c244350a4db077e085cf
11,2020-01-31T23:56:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,24c4ff81022e653f45cfb8bd0af3e276
18,2020-01-31T23:57:30+08:00,https://images.data.gov.sg/api/traffic-images/...,1709,1.313842,103.845603,480,640,d85d5d9db869044c3f64b54fa5fcb41d


In [60]:
Combined2020.to_csv('/content/drive/MyDrive/Project CV x Traffic/Data2020.csv', index=None)

# End of Part II
What a long Part. In this Part, you successfully made multiple API calls and obtained two months' worth of data.  

Next Part, we will retrieve the images from the URL we find in the DataFrame for both years. We will also prepare for setting up Google Colab for object detection with GPU.

The next part is quite long as well so prepare yourself.